In [ ]:
import os
import math
import random
import zipfile
import numpy as np
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
import albumentations as A
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
from sklearn.datasets import load_sample_image
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augmentation(image):
    #Random Crop
    shape = tf.shape(image)
    dims_factor = tf.random.uniform([], 0.5, 1.0, dtype=tf.float32)
    height_dim  = tf.multiply(dims_factor, tf.cast(shape[0], tf.float32))
    width_dim   = tf.multiply(dims_factor, tf.cast(shape[1], tf.float32))
    image = tf.image.random_crop(image, [height_dim, width_dim, 3])
    
    #Random Rotation by 90
    rotate_prob = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    if rotate_prob > .75:
        image = tf.image.rot90(image, k=3)
    elif rotate_prob > .5:
        image = tf.image.rot90(image, k=2)
    elif rotate_prob > .25:
        image = tf.image.rot90(image, k=1)
    
    #Random Rotation by n
    angle = random.uniform(0, 360)
    rotate_prob = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    max_angle   = angle*math.pi/180
    rotation    = tf.random.uniform(shape=[], minval=-max_angle, maxval=max_angle, dtype=tf.float32)
    image = tfa.image.rotate(image, rotation, interpolation = "BILINEAR")
    image = tf.image.central_crop(image, central_fraction=0.8)
    
    #Random flipping by n
    flip_prob = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
    if flip_prob > 0.75 :
        image = tf.image.transpose(image)
    elif flip_prob > 0.4:
        image = tf.image.random_flip_up_down(image)
    elif flip_prob > 0.3:
        image = tf.image.random_flip_left_right(image)
    
    #Random Lightning
    image = tf.image.random_brightness(image, 0.2,)
    image = tf.image.random_contrast(image, 0.6, 1.4,)
    image = tf.image.random_hue(image, 0.0)
    image = tf.image.random_saturation(image, 0.5, 1.5,)
    return image

def generate_augmented_images(input_dir, output_dir, target_size=(224, 224), num_augmentations=200):  
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            img_path = os.path.join(root, file)
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            for i in tqdm(range(num_augmentations), desc=f"Processing {file}"):
                augmented_image = augmentation(img_array[0])
                image_name = f"augmented_image_{file.split('_')[1].split('.')[0]}_{i}.jpg"
                image_path = os.path.join(output_dir, image_name)
                tf.keras.preprocessing.image.save_img(image_path, augmented_image)
                
def rename_files(folder_path):
    file_names = os.listdir(folder_path)
    counter = 1
    for file_name in tqdm(file_names, desc="Renaming files"):
        new_file_name = f"augmented_image_{counter:02d}.jpg"
        os.rename(os.path.join(folder_path, file_name), os.path.join(folder_path, new_file_name))
        counter += 1
        
def zip_files(source_folder, output_directory, zip_filename):
    try:
        if not os.path.exists(source_folder):
            raise FileNotFoundError(f"The folder '{source_folder}' does not exist.")
        if not os.path.exists(output_directory):
            os.makedirs(output_directory)
        files_to_zip = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
        zip_path = os.path.join(output_directory, zip_filename)
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for file in files_to_zip:
                file_path = os.path.join(source_folder, file)
                zipf.write(file_path, arcname=file)
        print(f"Files zipped successfully to '{zip_path}'.")
    except Exception as e:
        print(f"Error zipping files: {e}")
    try:
     files = os.listdir(source_folder)
     for file in files:
       file_path = os.path.join(source_folder, file)
       if os.path.isfile(file_path):
         os.remove(file_path)
     print("All files deleted successfully.")
    except OSError:
     print("Error occurred while deleting files.")
print("Success")

In [ ]:
if __name__ == "__main__":
    input_dir = "/kaggle/input/train-data"
    output_dir = "/kaggle/working/"
    directory = '/kaggle/working/augmented_data'
    zip_filename = 'augmented_data'
    
    generate_augmented_images(input_dir, output_dir, num_augmentations=500)
    rename_files(output_dir)
    zip_files(output_dir, directory, zip_filename)